In [1]:
import tensorflow as tf
tf.debugging.set_log_device_placement(True)
tf.device('/device:GPU.0')
a = tf.constant([[1, 2, 3],[4,5,6]])

b = tf.constant([[1, 2], [3,4], [5,6]])
c = tf.matmul(a,b)
print(c)
tf.__version__

Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op MatMul in device /job:localhost/replica:0/task:0/device:CPU:0
tf.Tensor(
[[22 28]
 [49 64]], shape=(2, 2), dtype=int32)


'2.10.0'

## Images path
Point to the folder where the images are

In [2]:
images_path = "../../../ImagenesPractica/xview_recognition/"

In [3]:
import uuid
import numpy as np

class GenericObject:
    """
    Generic object data.
    """
    def __init__(self):
        self.id = uuid.uuid4()
        self.bb = (-1, -1, -1, -1)
        self.category= -1

class GenericImage:
    """
    Generic image data.
    """
    def __init__(self, filename):
        self.filename = filename
        self.tile = np.array([-1, -1, -1, -1])  # (pt_x, pt_y, pt_x+width, pt_y+height)
        self.objects = list([])

    def add_object(self, obj: GenericObject):
        self.objects.append(obj)

## Define categories
Define the categories of the images (DON'T MODIFY)

In [4]:
categories = {13: 'CARGO_PLANE', 15: 'HELICOPTER', 18: 'SMALL_CAR', 19: 'BUS', 23: 'TRUCK', 41: 'MOTORBOAT', 47: 'FISHING_VESSEL', 60: 'DUMP_TRUCK', 64: 'EXCAVATOR', 73: 'BUILDING', 86: 'STORAGE_TANK', 91: 'SHIPPING_CONTAINER'}

## Transformations for data augmentation
Operations of rotation and flip.  
For each image we will rotate 90 degrees 3 times, then we will flip the original image and rotate it 90 degrees another 3 times more.  
For each image we will get 8 more.

In [5]:
def rotate_left(img):
    rotated_image = cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)
    return rotated_image

def rotate_right(img):
    rotated_image = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
    return rotated_image

def rotate_180(img):
    rotated_image = cv2.rotate(img, cv2.ROTATE_180)
    return rotated_image

def flip_image(img):
    flipped_image = cv2.flip(img, 1)
    return flipped_image

In [6]:
def save_image(my_image, filename):
    cv2.imwrite(images_path+filename, my_image)

## Code to perform data augmentation for an image
This code will create the 8 different images and also create the json data for each transformation

In [7]:
import cv2

def create_img_json(base_image_id, base_file_name, new_ending):
    new_img_json = {
        'id': base_image_id + new_ending,
        'file_name': base_file_name + new_ending + '.tif',
        'width': 224, 
        'height': 224
    }
    return new_img_json

def create_ann_json(base_image_id, category_id, new_ending):
    new_ann_json = {
        'id': 1,
        'image_id': base_image_id + new_ending,
        'category_id': category_id,
        'bbox': [0, 0, 224, 224], 
        'iscrowd': 0
    }
    return new_ann_json

def create_new_images(json_img, json_ann):
    new_json_imgs = []
    new_json_anns = []
    
    base_image_id = json_img['id']
    base_file_name = json_img['file_name'].replace(".tif","")
    category_id = json_ann['category_id']
    
    # Read original image
    original_image = cv2.imread(images_path+base_file_name+".tif")
    
    # Rotate image left and save image and create new json data
    rotated_left_image = rotate_left(original_image)
    save_image(rotated_left_image, base_file_name+'rli.tif')
    rli_img_json = create_img_json(base_image_id, base_file_name, 'rli')
    rli_ann_json = create_ann_json(base_image_id, category_id, 'rli')
    new_json_imgs.append(rli_img_json)
    new_json_anns.append(rli_ann_json)
    
    # Rotate image right and save image and create new json data
    rotated_right_image = rotate_right(original_image)
    save_image(rotated_right_image, base_file_name+'rri.tif')
    rri_img_json = create_img_json(base_image_id, base_file_name, 'rri')
    rri_ann_json = create_ann_json(base_image_id, category_id, 'rri')
    new_json_imgs.append(rri_img_json)
    new_json_anns.append(rri_ann_json)
    
    # Rotate image 180 degrees and save image and create new json data
    rotated_180_image = rotate_180(original_image)
    save_image(rotated_180_image, base_file_name+'rii.tif')
    rii_img_json = create_img_json(base_image_id, base_file_name, 'rii')
    rii_ann_json = create_ann_json(base_image_id, category_id, 'rii')
    new_json_imgs.append(rii_img_json)
    new_json_anns.append(rii_ann_json)
    
    # Flip and save image and create new json data
    flipped_image = flip_image(original_image)
    save_image(flipped_image, base_file_name+'fi.tif')
    fi_img_json = create_img_json(base_image_id, base_file_name, 'fi')
    fi_ann_json = create_ann_json(base_image_id, category_id, 'fi')
    new_json_imgs.append(fi_img_json)
    new_json_anns.append(fi_ann_json)
    
    # Rotate fliped left and save image and create new json data
    flipped_rotated_left_image = rotate_left(flipped_image)
    save_image(flipped_rotated_left_image, base_file_name+'fli.tif')
    fli_img_json = create_img_json(base_image_id, base_file_name, 'fli')
    fli_ann_json = create_ann_json(base_image_id, category_id, 'fli')
    new_json_imgs.append(fli_img_json)
    new_json_anns.append(fli_ann_json)
    
    # Rotate flipped right and save image and create new json data
    flipped_rotated_right_image = rotate_right(flipped_image)
    save_image(flipped_rotated_right_image, base_file_name+'fri.tif')
    fri_img_json = create_img_json(base_image_id, base_file_name, 'fri')
    fri_ann_json = create_ann_json(base_image_id, category_id, 'fri')
    new_json_imgs.append(fri_img_json)
    new_json_anns.append(fri_ann_json)
    
    # Rotate flipped 180 degrees and save image and create new json data
    flipped_rotated_180_image = rotate_180(flipped_image)
    save_image(flipped_rotated_180_image, base_file_name+'fii.tif')
    fii_img_json = create_img_json(base_image_id, base_file_name, 'fii')
    fii_ann_json = create_ann_json(base_image_id, category_id, 'fii')
    new_json_imgs.append(fii_img_json)
    new_json_anns.append(fii_ann_json)
    
    return new_json_imgs, new_json_anns

## Load our data base
Be sure to pick the correct json. DO THIS WITH **train.json**

In [8]:
import json
# Load database
json_file = images_path + 'xview_ann_train.json'
with open(json_file) as ifs:
    json_data = json.load(ifs)
ifs.close()

## Let's take a look at our data
We have a lot of images for SMALL_CAR, BUS, TRUCK, and BUILDING. We need more of the rest of categories.  
The output of this cell shows the frequency of each class **before** data augmentation.

In [9]:
import json
import numpy as np
counts = dict.fromkeys(categories.values(), 0)
for json_img, json_ann in zip(json_data['images'], json_data['annotations']):
    obj = GenericObject()
    obj.category = list(categories.values())[json_ann['category_id']-1]
    counts[obj.category] += 1

print(counts)

{'CARGO_PLANE': 628, 'HELICOPTER': 49, 'SMALL_CAR': 195133, 'BUS': 6549, 'TRUCK': 10640, 'MOTORBOAT': 1231, 'FISHING_VESSEL': 736, 'DUMP_TRUCK': 1238, 'EXCAVATOR': 706, 'BUILDING': 283491, 'STORAGE_TANK': 1462, 'SHIPPING_CONTAINER': 1522}


## Data augmentation
We will perform data augmentation now. Don't run this more than once.

In [10]:
import json
import numpy as np
counts = dict.fromkeys(categories.values(), 0)
anns = []
new_images = []
new_annotations = []
for json_img, json_ann in zip(json_data['images'], json_data['annotations']):
    obj = GenericObject()
    obj.category = list(categories.values())[json_ann['category_id']-1]
    
    # Data augmentation for categories that have low images
    if obj.category not in ['SMALL_CAR', 'BUS', 'TRUCK', 'BUILDING']:
        print(json_img['file_name'])
        augmented_images, augmented_annotations = create_new_images(json_img, json_ann)
        new_images += augmented_images
        new_annotations += augmented_annotations
    counts[obj.category] += 1
    
json_data['images'] += new_images
json_data['annotations'] += new_annotations

json_file = images_path + 'xview_ann_train_augmented.json'
with open(json_file, "w") as file:
    json.dump(json_data, file)

train/1072/597435c6-14e1-4f00-95da-c6d59f302e8f.tif
train/1072/1d471153-94ab-45fb-95fb-33e82e32e064.tif
train/2293/ba496026-8864-4e9a-a518-3e5cf9d8ae9b.tif
train/2293/911c2ab5-fd81-462f-9095-72b26e73b413.tif
train/2293/c15f3a8d-ff29-4edc-b303-b111009159ab.tif
train/2293/76d2850a-5617-4f4b-b44f-2e070d91b213.tif
train/2293/e011e83f-4a67-40b9-b664-eba2ba346483.tif
train/2293/89ccb8b6-71a8-4a38-93dc-30850d8a21aa.tif
train/2293/2ac096f5-6329-48d9-b7bd-8d574bc4a8f8.tif
train/2293/646b840f-8ef0-4230-905e-58b0a1d49f9d.tif
train/2293/c99fbc8b-afe4-455b-ade5-be6fb3225d54.tif
train/2293/8c809a67-23f3-49e6-8538-963ebc641762.tif
train/2293/d0b574cf-3417-4464-b02e-0ead1f348da8.tif
train/2293/fc185977-9cc1-4ede-b083-a892d10729cd.tif
train/2293/c16249fe-35b4-4b08-b589-83515a4e1ae3.tif
train/2293/74e4cc3d-a947-4da8-aff7-33eefe3f6401.tif
train/2293/80954d5f-ace8-40ae-ac5b-6111a6f7cd80.tif
train/2293/ab66b4ff-381e-4be5-9947-0aa0b681acf7.tif
train/2293/27645fcc-e04b-418e-832e-c398e4ec3a11.tif
train/2293/8

train/1820/8437842a-f80c-4e67-a865-f4488e2e904b.tif
train/1820/b1818574-db1c-4f1b-b053-9d3677c128a7.tif
train/1820/465bf667-ff99-4121-a2a7-77bbeefe2c0f.tif
train/1820/7946d9ca-26c7-44f6-bb2b-1901dee1092f.tif
train/1820/23bfd914-d729-4038-a033-6ba4bbddaf8e.tif
train/1820/3d37f189-8004-4fd1-a466-902538cacfac.tif
train/1820/23674c1e-9acc-45a8-bd40-040b3a006776.tif
train/1820/68471ed5-17f5-4d01-898d-af5333eddc13.tif
train/1820/b2dc04b8-17b5-42ac-83e7-c4e005385d72.tif
train/1820/20ffdb20-b8af-42a2-9b4a-9746345724d7.tif
train/1820/ffdd5494-afcf-4fd8-8926-3afc0f75b1b7.tif
train/1820/f7740758-e5ad-4672-b117-21101747f8f8.tif
train/1820/0d247040-f12f-4f50-9b8f-f57c8f4a87f6.tif
train/1820/b1f707cd-ca13-4594-aa64-5c3b9a3a251b.tif
train/1820/77d68608-7ec0-4ec5-8a3f-0912e1debeb2.tif
train/1820/e3e8f14e-45fe-487f-9cdf-615a5cba8ad8.tif
train/1820/ea3ee050-527a-4557-b2ed-1eb4a94a6941.tif
train/1820/cd7a9399-81d3-44d8-a5af-53035dcc6321.tif
train/1820/beaf9f47-cdd1-4108-bb60-e7b400f04caa.tif
train/1820/6

train/1695/20046a8a-ba81-44a0-9ee2-705a9bedc8e5.tif
train/1695/6546886b-e758-4745-9a0a-6d86cecb9be7.tif
train/1695/0d5d745d-9c13-48ba-ac44-83b7b4c41be7.tif
train/1695/215a1319-5840-4ed0-9092-90a66c0403d2.tif
train/1695/50124005-12fe-44e7-91f4-8912556d9d7f.tif
train/1695/d17c8271-bf8b-4d8d-ae60-19a3e6cf42ac.tif
train/1695/bd9968bb-b9af-44ea-bd74-eed63df5eccc.tif
train/1695/9e02c578-236d-43ef-b4bf-fbca37618abf.tif
train/1695/a34200d4-336b-4db2-be60-981d3563108d.tif
train/1695/d8a96da2-abcb-4a77-b717-22d5ac5c4e4a.tif
train/1695/43ccd3e9-086a-4f65-b2bd-1aacb37d2e67.tif
train/1695/f1826470-f871-475d-81d8-a20c05b940b5.tif
train/1695/e9aa1c93-63c8-4ec6-8640-26abd75f1bbd.tif
train/1695/b140cdea-f38c-4c2c-8619-fb163b8953fd.tif
train/1695/8a280f46-fce7-4732-9907-c3fa5908df42.tif
train/1695/e5b3d7c1-d6f6-4e69-a63d-12c7e0115edf.tif
train/1695/4eb505c2-a6d8-417b-83cb-5c515d8a5804.tif
train/1695/a0f6127c-216e-48a0-887e-6e274450edb2.tif
train/1695/eb2314ea-349d-4353-a035-8f33cd203b99.tif
train/1695/c

train/1444/2180406b-de44-44a2-b30c-504e72555706.tif
train/1444/17e9422d-bec7-45d8-a077-88a4c3cf29b5.tif
train/1444/1057d2f5-6a0a-47f7-bdea-5262eaff0dad.tif
train/1444/a3d01eda-9b70-42d3-b2f6-8393c8092bbe.tif
train/1444/d24ac1a7-d054-4b1e-ba47-801447d9a35f.tif
train/1444/8e462362-7a1c-4d46-b7dd-7bd4b30f2d1f.tif
train/1444/a3b44ecb-1f93-460f-a3d5-4b4de1b8bf94.tif
train/1444/a86cd61e-57d5-4273-8e10-f50c613464ba.tif
train/1633/1aa5754a-4962-44d7-a288-9d774d938d39.tif
train/1633/a809f00a-265e-4aca-89ff-d4821c7abc09.tif
train/1633/e25015d6-7359-4125-a2a1-af148c9a6929.tif
train/1633/df2b309c-cddc-426a-8c37-9a9d49f09d22.tif
train/1633/b39034df-d1e7-4ada-b2a3-9d15ea115dc5.tif
train/1633/252302c2-22db-4f50-b450-ad849c74aceb.tif
train/1633/b68feec0-21a4-4720-9c69-71860f97f640.tif
train/1633/46947f04-4f23-4ee2-9084-d7295f172a0c.tif
train/1633/462165a7-7ca7-4172-9829-c35aeedaa784.tif
train/1633/56828046-576e-4e0c-8608-50ad30d9b7bb.tif
train/1633/62d34f10-9f92-4cab-8c89-b1626ee6bdca.tif
train/1633/5

train/1158/acf9a704-7f37-4225-8b21-167d29a509c2.tif
train/1158/cafe4f2f-9c2c-42da-84f9-1171fc8e67d9.tif
train/1158/519bc8f1-3050-4071-b654-e2861edd70c2.tif
train/1158/0a6cf1ef-af55-4149-bcf2-63634bb4f575.tif
train/1158/f55c8159-0c4e-488b-8191-dec40798c2a9.tif
train/1158/95e94086-a6fa-4bb1-89a3-490c7d1020fa.tif
train/1158/2dc365de-a0e2-4758-a308-988cf81f5b2e.tif
train/1158/ab131624-c000-468c-8bc8-dab09d868abc.tif
train/1158/7045146e-8169-4a75-ae49-1bab1f0e58e0.tif
train/1158/a4c37813-d521-4712-be7d-66b1cf247cc0.tif
train/1158/1a06ccec-d7c8-4e7e-8090-e4dd362a8ad2.tif
train/1158/288c9470-9789-46b5-9c6e-263045169aa5.tif
train/1158/c5faf169-c55e-4c00-b983-43c85a3325b4.tif
train/1158/a8bf57da-80f9-4529-b392-82b8b5da3005.tif
train/1158/ac82e9cc-5f8c-4ac9-a713-de2c75935230.tif
train/1158/cb37e2b5-5872-4d15-9c47-170ce1a7c8b6.tif
train/1158/e51ef0ba-f11c-40d0-9a37-7cb20c62e0d8.tif
train/1158/662f84b7-e1f7-4a1d-9c21-9fe5c9cb886d.tif
train/1158/1d3371f2-5408-4e4d-9709-1de1069efd39.tif
train/1158/f

...

train/283/da67b8b6-17fe-43d8-a323-4699e4acc137.tif
train/283/1613d804-8899-474e-b065-324e4b501dd2.tif
train/1042/90b99a0b-1323-4775-972e-da9e50943f19.tif
train/1042/668fb8d4-05fa-413e-8eb1-539c09e18814.tif
train/1042/b62416fe-0910-4ea0-b4cc-9ac99d2b3ad0.tif
train/1042/4145b1d9-9747-443f-a083-68e475f5d120.tif
train/1042/875ab83d-cf0b-471d-a874-385501095450.tif
train/1042/03db3cb1-8862-407a-9ce7-d71b59093f50.tif
train/1042/c4a3769a-ff9d-4466-97d3-4624e7aa78df.tif
train/1042/679c038b-1c14-4c1f-be89-4a0c7434467b.tif
train/1042/2b225761-8754-46fa-b87d-7decc68d8339.tif
train/1042/7881be8f-4d97-4c30-8416-f127c7593cff.tif
train/1042/58fedc08-7732-476f-9d02-43984f96007d.tif
train/1042/e149e998-dbdc-4bf5-ae56-13ee6856a68d.tif
train/1042/31f82c14-5969-4344-90b3-79834786bc31.tif
train/1042/1e4508ff-a86b-4867-8409-9771c8f3e007.tif
train/1042/fbf77445-3f44-48ca-8f2a-ce291830bfa4.tif
train/1042/de04697d-7c8a-48b5-9816-1e7eba2076d5.tif
train/1042/87a09805-2267-49cb-86a8-29dc67e02a84.tif
train/1042/05a

train/24/350646be-1d9f-4794-894f-cc4b1ef7c0cd.tif
train/24/4779cf4c-3f48-4002-9f11-379995712141.tif
train/24/6faac0fa-f9cf-424f-bc70-d184296281a8.tif
train/24/edcff6f1-366a-440c-9481-66b3e76237bb.tif
train/24/64816385-24b4-499e-8493-a81410d2f1f0.tif
train/24/359bbf63-b60d-4a75-85d6-a7c9b54cff19.tif
train/24/234fd4b6-8a9a-4869-a4c1-e1821f169821.tif
train/24/e15a7875-5ba2-4013-8e37-44a898204561.tif
train/24/0d0d6c0e-d6b3-436f-8582-c017dca46f6f.tif
train/24/45397c97-4902-45d0-bdcc-32d70106b798.tif
train/24/73b1c348-bb48-48fd-9776-64d47857a0f0.tif
train/24/1d9ebb44-7491-4199-bc98-b6b5d7fbfa7b.tif
train/24/780455aa-905b-41fc-a81e-e8538a58c0e6.tif
train/774/06704e9b-e049-41bd-a0bf-7e7867c36432.tif
train/1061/9af104af-3751-4f75-9bdd-94c18be76490.tif
train/1180/1c1b2db6-ff37-4757-9131-557bc37d0ffe.tif
train/1180/e0181e98-644d-41b7-8b6c-3cbe6730f8ea.tif
train/610/8b4dec60-4860-4b1d-a18a-5329a94b3f3e.tif
train/500/c7c79df7-3876-4b34-8978-21851f7b728c.tif
train/500/eb240bc9-79fc-4a43-ade1-cbb67f5

train/1694/47a6f9b2-ff5c-4abb-9c8d-1fe3146d24d3.tif
train/1985/5cc45b73-3167-4424-8362-05d2477a86f2.tif
train/1985/07dfb9a2-8253-4bf4-a577-362050ce8d21.tif
train/1985/a7c0bb0e-bf62-45aa-a72a-6b6706a7524f.tif
train/1985/f6882b33-82ee-436c-b68d-3ebf484097d9.tif
train/1985/28058c0e-5c46-4edc-9488-cb60779e0374.tif
train/531/3c39dd96-cc55-4b2d-b9c7-c57831f0ec00.tif
train/531/5584c16a-97dd-483f-bf28-aea9956f5924.tif
train/531/bea77405-c30a-4c15-b129-ed7528753df9.tif
train/531/b2657b81-2945-4ced-b7ee-cbb861fcb49c.tif
train/1829/c09d274a-55ff-4133-bfd5-be474c6b8d65.tif
train/2423/bd28cd3e-d28f-4f06-b32e-339f09b717aa.tif
train/2423/b3fcb6bf-564f-417d-b175-4bb638b98e9b.tif
train/2423/d9b4790a-7bd8-438d-a525-f8d4f215d119.tif
train/2423/533725bd-5909-419d-bfe9-22813e96afda.tif
train/2423/23b0ea34-967d-4ae3-b1e6-076dd536f07e.tif
train/2423/ed316100-f95a-4d30-82f3-8067e4a07dde.tif
train/2423/4b5122eb-73dd-4487-9ea3-a3f0cffb5445.tif
train/2423/eca5675c-f517-45c2-821b-993c36c2c8be.tif
train/2423/7f1e9

train/73/28c0ce8a-0084-4064-939f-6a091a79e5ca.tif
train/73/cdddfacd-875c-41fc-9af3-60805812be18.tif
train/73/d5608e21-8515-4324-984c-b9e47aa66569.tif
train/73/cc6a9655-3973-46af-bc6d-c984f30e8757.tif
train/73/c1f5588a-dfef-4979-808f-e2c4116566cf.tif
train/73/0aa3fd30-8628-46c9-90b8-963488afc089.tif
train/73/331a7f49-6ebf-480d-8cf9-8cf9564f79f1.tif
train/73/80acd1d8-e46d-445e-b983-4f7df7d244cb.tif
train/73/e98ac2d2-63da-494a-8cc1-d57bba9d9609.tif
train/73/eda4f36c-6389-4711-81e5-03e3ee4d2da2.tif
train/73/bff9c36e-0e94-465e-9c58-cfd0a4a3ee3e.tif
train/73/a19cb608-d779-46b3-be4f-9f62f35db65e.tif
train/73/6b0a3a15-aee6-44ec-93f0-b3802862cd69.tif
train/73/bd0b1fbc-1e26-48cf-9747-ae826ee89c26.tif
train/73/9f71290f-1271-4762-bc5f-2b258482481d.tif
train/73/9fcbf602-e7f2-481f-bab2-b0a213855ecb.tif
train/73/c59bdbe0-b045-41f1-8f28-2b293ec2103f.tif
train/73/2dcd6c98-8668-49fa-8130-553cbf7a4bb5.tif
train/73/9c3afbec-99de-4e57-a02d-736c4ed797d3.tif
train/73/d7298752-d8a2-47a3-bafc-b7c20916713c.tif


train/805/4cb9024c-0fd8-43d1-b5d8-fcc614d36e3b.tif
train/805/2a35415f-72fc-4a55-8d3a-84f70e6488cc.tif
train/805/b3f8d2f1-a3cb-4872-9317-cd0e436e9074.tif
train/805/204a915f-fd24-42ea-9fa3-5ac5f0b06bc7.tif
train/805/29bbb313-9da6-4919-b0e1-bd342b3ba9c5.tif
train/805/2254609b-65ba-427a-b923-0481aa13d0f7.tif
train/805/b4001221-7574-4507-869f-6fe2bd49df13.tif
train/805/991095b3-c33c-4427-9aa8-713d08ce21fd.tif
train/129/3b4d1255-5e7e-4ce0-8387-a94aaa5de469.tif
train/129/c2ddd369-cece-4c14-81c0-58d6a2ec5caf.tif
train/129/b6e4e9ca-d6a2-49dd-b44c-b3e61358eb98.tif
train/129/7076ed6b-0d52-4ae0-ad10-9aaa4010f4ea.tif
train/129/fc313e3d-81ba-45e0-a3d1-dcb972456686.tif
train/362/bc84b6d9-2b51-4ae8-a351-9c4b5a6ceafe.tif
train/362/b3b608c2-95d9-48b5-8a57-b0401929122f.tif
train/822/c705f0ae-86bb-4651-a8da-bb5ae6e17e54.tif
train/2584/bcee7007-31da-4fba-ae3b-efbb2157e988.tif
train/2584/df8574bb-9f53-4f27-b104-befcd25a8d81.tif
train/2584/5ac17a39-36b1-41c4-8a86-d9f93583b323.tif
train/125/1c934344-a73d-40a7

## Results
In the below cell the new frequencies are shown after the data augmentation

In [11]:
import json
import numpy as np
    
counts = dict.fromkeys(categories.values(), 0)
for json_img, json_ann in zip(json_data['images'], json_data['annotations']):
    obj.category = list(categories.values())[json_ann['category_id']-1]
    counts[obj.category] += 1

print(counts)

{'CARGO_PLANE': 5024, 'HELICOPTER': 392, 'SMALL_CAR': 195133, 'BUS': 6549, 'TRUCK': 10640, 'MOTORBOAT': 9848, 'FISHING_VESSEL': 5888, 'DUMP_TRUCK': 9904, 'EXCAVATOR': 5648, 'BUILDING': 283491, 'STORAGE_TANK': 11696, 'SHIPPING_CONTAINER': 12176}
